In [40]:
# Import necessary libraries and modules
import pandas as pd
import datetime as dt 
import yfinance as yf
import json
from requests import Session
import requests
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass
import requests_cache

### GET STOCK DATA

In [10]:
# Session Limiter for the API call to Yahoo Finance to prevent timing out during function call
session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  #max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

In [71]:
# Current list of Standard & Poor 500 companies from Wikipedia: 
sp500_companies = pd.read_csv("/Users/rosario/Code/CodeOp/Group-Project/sp500_companies.csv")

In [11]:
# Function to scrape the historical data including Adjusted Closing Price from Yahoo finance
# Return a data frame with Date as the index

def get_ticker_history(df, period=" "):
    ticker_data = {}
    
    # Iterate through each row of the DataFrame
    for index, row in df.iterrows():
        ticker = row['Symbol'] 
        print("Fetching data for ticker:", ticker)  # Print the ticker being processed
        # Fetch historical data for the ticker
        history_data = yf.download(tickers=ticker, period=period)
        # Add the ticker symbol as a feature in the historical data DataFrame
        history_data['Symbol'] = ticker
        # Store the historical data in the dictionary
        ticker_data[ticker] = history_data
        
    # Concatenate historical data for all tickers into a single DataFrame
    combined_data = pd.concat(ticker_data.values())
    combined_data.index.name = 'Date'  # Set the name of the index to 'Date'
    
    return combined_data

In [ ]:
# Run the function to call the API and start the data scraping
sp500_history = get_ticker_history(sp500_companies, period="max")

In [80]:
# sp500_history has over 4 million rows
sp500_history.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4224446 entries, 1962-01-02 to 2024-05-08
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Open       float64
 1   High       float64
 2   Low        float64
 3   Close      float64
 4   Adj Close  float64
 5   Volume     float64
 6   Symbol     object 
dtypes: float64(6), object(1)
memory usage: 257.8+ MB


In [95]:
# Subset the data frame to the time frame of 10 years
# Rename the data frame from sp500_history to sp500_adj_df
sp500_adj_df = sp500_history.loc['2014-04-01':'2024-04-30']

In [120]:
# Save to CSV to share with the team
sp500_adj_df.to_csv('sp500_adj_df.csv', index=True)

In [4]:
# Read CSV to start financial data EDA
sp500_adj_df = pd.read_csv("/Users/rosario/Code/CodeOp/Group-Project/sp500_adj_df.csv")

In [96]:
sp500_adj_df.head()

,Open,High,Low,Close,Adj Close,Volume,Symbol
Date,,,,,,,
2014-04-01,113.612038,114.255852,113.436455,114.155518,76.269997,2835477.0,MMM
2014-04-02,113.704010,113.921402,113.152176,113.712372,75.973930,3924554.0,MMM
2014-04-03,113.896324,114.707359,113.461540,113.829430,76.052109,3200735.0,MMM
2014-04-04,114.464882,115.192307,113.520065,113.595322,75.895714,3271419.0,MMM
2014-04-07,113.553513,113.921402,112.366219,112.374580,75.080109,3357770.0,MMM


In [101]:
sp500_adj_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1226702 entries, 2014-04-01 to 2024-03-28
Data columns (total 7 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   Open       1226702 non-null  float64
 1   High       1226702 non-null  float64
 2   Low        1226702 non-null  float64
 3   Close      1226702 non-null  float64
 4   Adj Close  1226702 non-null  float64
 5   Volume     1226702 non-null  float64
 6   Symbol     1226702 non-null  object 
dtypes: float64(6), object(1)
memory usage: 74.9+ MB


In [17]:
# Convert 'Date' column to datetime
sp500_adj_df['Date'] = pd.to_datetime(sp500_adj_df['Date'])

In [26]:
# Set 'Date' column as the index
sp500_adj_df.set_index('Date', inplace=True)

In [103]:
sp500_adj_df["Symbol"].unique()

array(['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL',
       'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT',
       'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP',
       'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI',
       'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM',
       'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB',
       'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX',
       'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA',
       'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BLDR', 'BG', 'CDNS',
       'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT',
       'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF',
       'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI',
       'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO',
       'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG',
     

In [104]:
sp500_adj_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1226702 entries, 2014-04-01 to 2024-03-28
Data columns (total 7 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   Open       1226702 non-null  float64
 1   High       1226702 non-null  float64
 2   Low        1226702 non-null  float64
 3   Close      1226702 non-null  float64
 4   Adj Close  1226702 non-null  float64
 5   Volume     1226702 non-null  float64
 6   Symbol     1226702 non-null  object 
dtypes: float64(6), object(1)
memory usage: 74.9+ MB


### GET ECONOMIC INDICATORS

#### GDP per capita - data returned is quarterly gdp per capita

In [68]:
# using Api key, scrape alphavantage.co GDP per capita data

url = 'https://www.alphavantage.co/query?function=REAL_GDP_PER_CAPITA&apikey=06N85DP7BER4MKDB'
r = requests.get(url)
gdp_data = r.json()

In [130]:
gdp_per_capita = pd.DataFrame(gdp_data['data']) # Create dataframe from json object
gdp_per_capita['value'] = gdp_per_capita['value'].astype(float) 
gdp_per_capita.set_index('date', inplace =True) # Set datetime as index
gdp_per_capita.rename(columns={'value': 'gdp'}, inplace=True) # Rename 'value' to 'gdp'
gdp_per_capita = gdp_per_capita.sort_index() # Sort dataframe from oldest to newest

In [129]:
gdp_per_capita.info()

<class 'pandas.core.frame.DataFrame'>
Index: 309 entries, 1947-01-01 to 2024-01-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   gdp     309 non-null    float64
dtypes: float64(1)
memory usage: 12.9+ KB


In [126]:
# Subset data frame to the last 10 years 

gdp_df = gdp_per_capita.loc['2014-01-01':'2024-03-31']

In [128]:
# Save as csv to share with team 

gdp_df.to_csv('gdp_df.csv', index=True)

#### Inflation - data returned is annual inflation rate

In [72]:
# using Api key, scrape alphavantage.co inflation data

url = 'https://www.alphavantage.co/query?function=INFLATION&apikey=06N85DP7BER4MKDB'
r = requests.get(url)
inflation_data = r.json()

In [132]:
inflation = pd.DataFrame(inflation_data['data']) # Create dataframe from json object
inflation['value'] = inflation['value'].astype(float)
inflation.set_index('date', inplace =True) # Set datetime as index
inflation.rename(columns={'value': 'inflation'}, inplace=True) # Rename 'value' to 'inflation'
inflation = inflation.sort_index() # Sort dataframe from oldest to newest

In [133]:
inflation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, 1960-01-01 to 2022-01-01
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   inflation  63 non-null     float64
dtypes: float64(1)
memory usage: 1008.0+ bytes


In [134]:
# Subset data frame to the last 10 years 

inflation_df = inflation.loc['2014-01-01':'2024-03-31']

In [137]:
# Save as csv to share with team 

inflation_df.to_csv('inflation_df.csv', index=True)

#### Unemployment Rate - data returned is monthly unemployment data

In [91]:
# using Api key, scrape alphavantage.co unemployment rate data

url = 'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey=06N85DP7BER4MKDB'
r = requests.get(url)
unemployment_data = r.json()

In [138]:
# unemployment = pd.DataFrame(unemployment_data['data']) # Create dataframe from json object
# unemployment['value'] = unemployment['value'].astype(float)
# unemployment.set_index('date', inplace =True) # Set datetime as index
# unemployment.rename(columns={'value': 'unemployment'}, inplace=True) # Rename 'value' to 'unemployment'
unemployment = unemployment.sort_index() # Sort dataframe from oldest to newest

In [140]:
unemployment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 916 entries, 1948-01-01 to 2024-04-01
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   unemployment  916 non-null    float64
dtypes: float64(1)
memory usage: 14.3+ KB


In [141]:
# Subset data frame to the last 10 years 

unemployment_df = unemployment.loc['2014-01-01':'2024-03-31']

In [145]:
# Save as csv to share with team 

unemployment_df.to_csv('unemployment_df.csv', index=True)

#### UNUSED CODE, but may need later

In [2]:
# GETTING TICKER DATA FROM CACHE:
# session = requests_cache.CachedSession('yfinance.cache')
# session.headers['User-agent'] = 'my-program/1.0'
# ticker = yf.Ticker('msft', session=session)
# # The scraped response will be stored in the cache
# ticker.actions

# READ ORIGINAL .CSV OF SCRAPED DATA WITHOUT ADJUSTED CLOSING PRICE: 
# sp500_df = pd.read_csv("/Users/rosario/Code/CodeOp/Group-Project/sp500_history.csv")

# PIVOT DATAFRAME "SYMBOLS" AS FEATURES: 
# def create_new_dataframe(df):
#     # Reset the index if 'date' is set as the index
#     if 'Date' not in df.columns:
#         df.reset_index(inplace=True)
    
#     # Pivot the DataFrame
#     new_df = df.pivot(index='Date', columns='Symbol', values='Close')
    
#     return new_df